### Experiments with Large Language Models

In [1]:
# General imports
import pandas as pd
import numpy as np
from tqdm import tqdm

#  PyTorch imports
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn

# Transformers imports
from transformers import T5Tokenizer, T5ForConditionalGeneration, PreTrainedTokenizer
from transformers import Trainer, TrainingArguments, AdamW

# Types
from typing import List, Dict, Tuple, Union

# Set seed
torch.manual_seed(42)
np.random.seed(42)



#### Data

In [2]:
wiki = []
with open('data/wiki.txt', 'r', encoding='utf8') as f:
    for line in f:
        wiki.append(line)

wiki = pd.DataFrame(wiki, columns=['text'])
wiki.head()

,text
0,Khatchig Mouradian. Khatchig Mouradian is a jo...
1,Jacob Henry Studer. Jacob Henry Studer (26 Feb...
2,"John Stephen. Born in Glasgow, Stephen became ..."
3,Georgina Willis. Georgina Willis is an award w...
4,Stanley Corrsin. Corrsin was born on 3 April 1...


In [3]:
# Read tsv file
birth_places_train = pd.read_csv('data/birth_places_train.tsv', sep='\t') 
birth_places_train.head()

,Where was Khatchig Mouradian born?,Lebanon
0,Where was Jacob Henry Studer born?,Columbus
1,Where was John Stephen born?,Glasgow
2,Where was Georgina Willis born?,Australia
3,Where was Stanley Corrsin born?,Philadelphia
4,Where was Eduard Ender born?,Rome


In [4]:
birth_places_test = pd.read_csv('data/birth_places_test.tsv', sep='\t')
birth_places_test.head()

,Where was Bryan Dubreuiel born?,Atlanta
0,Where was Ralf Wadephul born?,Berlin
1,Where was Joseph Baggaley born?,England
2,Where was Sandhya Sanjana born?,Mumbai
3,Where was Alfred Mele born?,Detroit
4,Where was Murray Esler born?,Geelong


### Build a single file dataset with $(q, c, a)$ structure

In [5]:
def build_dataset_file(
    qa_df: pd.DataFrame, # questions and answers
    wiki_df: pd.DataFrame, # context
    filename: str # name of the file to be created
) -> None:

    """
    Build dataset file for T5 training
    """
    qa_df_values = qa_df.values
    wiki_df_values = wiki_df.values

    matches = {}
    for i in tqdm(range(len(qa_df_values)), desc='Matching questions and contexts', total=len(qa_df_values)):
        [q, a] = qa_df_values[i]
        person = q.split(' ')[2:-1] # Get the name of the person
        person = ' '.join(person) # Join the name of the person

        # Find the context of the person
        for j in range(len(wiki_df_values)):
            c = wiki_df_values[j][0] # There is only one column in the wiki DataFrame
            if person in c:
                # Remove new line characters from context
                c = c.replace('\n', '')
                matches[person] = (q, c, a)
                break
    
    print(f'Number of entries preprocessed: {len(matches.keys())}')

    # Build the dataset file
    # Create a new csv file with the columns: question, context, answer
    with open(f'data/{filename}.tsv', 'w', encoding='utf8') as f:
        for k in matches.keys():
            q, c, a = matches[k]
            # Write the question, context and answer in the file on the same line
            f.write(f'{q}\t{c}\t{a}\n')
            
    

In [6]:
build_dataset_file(birth_places_train, wiki, 'birth_places_train_clean')
build_dataset_file(birth_places_test, wiki, 'birth_places_test_clean')

Matching questions and contexts: 100%|██████████| 1999/1999 [00:00<00:00, 3008.66it/s]


Number of entries preprocessed: 1990


Matching questions and contexts: 100%|██████████| 499/499 [00:00<00:00, 1299.99it/s]


Number of entries preprocessed: 499


In [7]:
# Read the csv file
train_dataset_df = pd.read_csv('data/birth_places_train_clean.tsv', sep='\t', names=['question', 'context', 'answer'])
train_dataset_df.head()

,question,context,answer
0,Where was Jacob Henry Studer born?,Jacob Henry Studer. Jacob Henry Studer (26 Feb...,Columbus
1,Where was John Stephen born?,"John Stephen. Born in Glasgow, Stephen became ...",Glasgow
2,Where was Georgina Willis born?,Georgina Willis. Georgina Willis is an award w...,Australia
3,Where was Stanley Corrsin born?,Stanley Corrsin. Corrsin was born on 3 April 1...,Philadelphia
4,Where was Eduard Ender born?,Eduard Ender. Eduard Ender (3 March 1822 Rome ...,Rome


In [8]:
test_dataset_df = pd.read_csv('data/birth_places_test_clean.tsv', sep='\t', names=['question', 'context', 'answer'])
test_dataset_df.head()

,question,context,answer
0,Where was Ralf Wadephul born?,"Ralf Wadephul. Ralf Wadephul, born 1958 in Ber...",Berlin
1,Where was Joseph Baggaley born?,Joseph Baggaley. Joseph Baggaley (c. 1884 -- 1...,England
2,Where was Sandhya Sanjana born?,Sandhya Sanjana. Sandhya Sanjana (); is a sing...,Mumbai
3,Where was Alfred Mele born?,"Alfred Mele. Born in Detroit, Michigan, Mele a...",Detroit
4,Where was Murray Esler born?,Murray Esler. Professor Murray David Esler (bo...,Geelong


### Create utility functions for the $(q, c, a)$ scenario

In [9]:
# process the examples in input and target text format and the eos token at the end 
def add_eos_to_examples(entry: Tuple[str, str, str]) -> Tuple[str, str]:
    question, context, answer = entry
    result = {}
    result['input_text'] = 'question: %s  context: %s </s>' % (question, context)
    result['target_text'] = '%s </s>' % (answer)
    return result

# tokenize the examples
def convert_to_features(qa_entry: Tuple[str, str, str], tokenizer: PreTrainedTokenizer) -> Dict[str, List[int]]:
    input_encodings = tokenizer.encode_plus(text=qa_entry['input_text'], padding='max_length', max_length=512, truncation=True) # the context is limited to 512 tokens
    target_encodings = tokenizer.encode_plus(text=qa_entry['target_text'], padding='max_length', max_length=32, truncation=True) # the answer is limited to 32 tokens, which should conver most of the answers 

    encodings = {
        'input_ids': np.array(input_encodings['input_ids']), 
        'attention_mask': np.array(input_encodings['attention_mask']),
        'target_ids': np.array(target_encodings['input_ids']),
        'target_attention_mask': np.array(target_encodings['attention_mask'])
    }

    return encodings

In [10]:
# Test the functions on a sample
tokenizer = T5Tokenizer.from_pretrained('t5-small')
sample = train_dataset_df.sample(1)
sample = add_eos_to_examples(sample.values[0])
sample = convert_to_features(sample, tokenizer)

c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\models\t5\tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\models\t5\tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos

In [11]:
class BirthPlaceDataset(Dataset):
    """
    Dataset for birth place prediction made on a (q, c, a) format,
    where q is the question, c is the context and a is the answer.

    Args:
        df (pd.DataFrame): DataFrame containing the questions, contexts and answers
    """
    def __init__(self, df: pd.DataFrame) -> None:
        self.df = df
        self.df_values = self.df.values
        self.tokenizer = T5Tokenizer.from_pretrained('t5-small')

    def __len__(self) -> int:
        return len(self.df)
    
    def __getitem__(self, idx: int) -> Dict[str, List[int]]:
        entry = self.df_values[idx]
        entry = add_eos_to_examples(entry)
        entry = convert_to_features(entry, self.tokenizer)
        return entry

In [12]:
# Create two datasets, one for training and one for validation
train_dataset = BirthPlaceDataset(train_dataset_df)
val_dataset = BirthPlaceDataset(test_dataset_df)

# Check if each element in the list of batches has the same shape

# Check for input_ids
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['input_ids'].shape != (512,):
        print(f'Entry {i} has a different shape for input_ids: {entry["input_ids"].shape}')

# Check for attention_mask
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['attention_mask'].shape != (512,):
        print(f'Entry {i} has a different shape for attention_mask: {entry["attention_mask"].shape}')

# Check for target_ids
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['target_ids'].shape != (32,):
        print(f'Entry {i} has a different shape for target_ids: {entry["target_ids"].shape}')

# Check for target_attention_mask
for i in range(len(val_dataset)):
    entry = train_dataset[i]
    if entry['target_attention_mask'].shape != (32,):
        print(f'Entry {i} has a different shape for target_attention_mask: {entry["target_attention_mask"].shape}')

#### A sample from new the new dataset

In [13]:
print(f'Number of training examples: {len(train_dataset)}')
print(f'Number of validation examples: {len(val_dataset)}')

Number of training examples: 1990
Number of validation examples: 499


In [14]:
## Load t5 small model for question answering
model = T5ForConditionalGeneration.from_pretrained('t5-small', return_dict=True)

In [15]:
tokenizer = T5Tokenizer.from_pretrained('t5-small', return_dict=True)

#### Testing the t5 model

In [16]:
# Test model
input_ids = tokenizer.encode("translate English to German: How old are you?", return_tensors="pt")
outputs = model.generate(input_ids, max_length=40, num_beams=4, early_stopping=True)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Wie alt sind Sie?


#### [ Task 1 ] Cold evaluation of the pretrained T5-small model

The evaluation script was taken from SQuAD dataset and it validates the exact matches of the model answers compared with the real ones.

In [17]:
# Test dataloader
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8, shuffle=True)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Test is each batch has the correct shape
for batch in train_dataloader:
    input_ids = batch['input_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    target_ids = batch['target_ids'].to(device)
    target_attention_mask = batch['target_attention_mask'].to(device)
    assert input_ids.shape == (8, 512), f'input_ids has a shape of {input_ids.shape}'
    assert attention_mask.shape == (8, 512), f'attention_mask has a shape of {attention_mask.shape}'
    assert target_ids.shape == (8, 32), f'target_ids has a shape of {target_ids.shape}'
    assert target_attention_mask.shape == (8, 32), f'target_attention_mask has a shape of {target_attention_mask.shape}'
    break

#### Evaluation script

In [18]:
from collections import Counter
import string
import re


def normalize_answer(s: str) -> str:
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)


def evaluate(gold_answers, predictions):
    f1 = exact_match = total = 0

    for ground_truths, prediction in zip(gold_answers, predictions):
      total += 1
      exact_match += metric_max_over_ground_truths(
                    exact_match_score, prediction, ground_truths)
      f1 += metric_max_over_ground_truths(
          f1_score, prediction, ground_truths)
    
    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}

In [19]:
answers = []
for batch in tqdm(val_dataloader):
  outs = model.generate(input_ids=batch['input_ids'], 
                        attention_mask=batch['attention_mask'],
                        max_length=32,
                        early_stopping=True)
  outs = [tokenizer.decode(ids) for ids in outs]
  answers.extend(outs)

100%|██████████| 63/63 [03:15<00:00,  3.10s/it]


In [20]:
predictions = []
references = []
for ref, pred in zip(val_dataset, answers):
  a = ref['target_ids']  
  a = tokenizer.decode(a)

  # Remove padding
  a = a.replace('<pad>', '')
  a = a.replace('</s>', '')
  a = a.replace('<s>', '')
  
  # Remove all whitespace from the beginning and end
  a = a.rstrip()

  pred = pred.replace('<pad>', '')
  pred = pred.replace('</s>', '')
  pred = pred.replace('<s>', '')
  
  # Remove all whitespace
  pred.replace('', '')

  predictions.append(pred)
  references.append(a)
  

In [21]:
predictions[11], references[11]

(' Oxford, England', 'Washington')

In [173]:
evaluate(references, predictions)

{'exact_match': 0.0, 'f1': 0.10020040080160321}

#### Conclusion on preliminary results

TBD

#### [Task 2] Finetuning process

In [43]:
from transformers import EvalPrediction
from transformers import (
    HfArgumentParser,
    DataCollator,
    Trainer,
    TrainingArguments,
    set_seed,
    DefaultDataCollator,
)

import logging
from dataclasses import dataclass, field
from typing import Optional
import os

In [44]:
# Setup logging
logger = logging.getLogger(__name__)

##### Data collator 
- This is used to transform batches

In [45]:
@dataclass
class T5DataCollator(DefaultDataCollator):
    def collate_batch(self, batch: List) -> Dict[str, torch.Tensor]:
        """
        Take a list of samples from a Dataset and collate them into a batch.
        Returns:
            A dictionary of tensors
        """
        input_ids = torch.stack([example['input_ids'] for example in batch])
        lm_labels = torch.stack([example['target_ids'] for example in batch])
        lm_labels[lm_labels[:, :] == 0] = -100
        attention_mask = torch.stack([example['attention_mask'] for example in batch])
        decoder_attention_mask = torch.stack([example['target_attention_mask'] for example in batch])
        

        return {
            'input_ids': input_ids, 
            'attention_mask': attention_mask,
            'lm_labels': lm_labels, 
            'decoder_attention_mask': decoder_attention_mask
        }


In [46]:
@dataclass
class ModelArguments:
    """
    Arguments pertaining to which model/config/tokenizer we are going to fine-tune from.
    """

    model_name_or_path: str = field(
        metadata={"help": "Path to pretrained model or model identifier from huggingface.co/models"}
    )
    tokenizer_name: Optional[str] = field(
        default=None, metadata={"help": "Pretrained tokenizer name or path if not the same as model_name"}
    )
    cache_dir: Optional[str] = field(
        default=None, metadata={"help": "Where do you want to store the pretrained models downloaded from s3"}
    )

In [47]:

@dataclass
class DataTrainingArguments:
    """
    Arguments pertaining to what data we are going to input our model for training and eval.
    """
    train_file_path: Optional[str] = field(
        default='train_data.pt',
        metadata={"help": "Path for cached train dataset"},
    )
    valid_file_path: Optional[str] = field(
        default='valid_data.pt',
        metadata={"help": "Path for cached valid dataset"},
    )
    max_len: Optional[int] = field(
        default=512,
        metadata={"help": "Max input length for the source text"},
    )
    target_max_len: Optional[int] = field(
        default=32,
        metadata={"help": "Max input length for the target text"},
    )

Configure logger

In [48]:
logging.basicConfig(
        format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
        datefmt="%m/%d/%Y %H:%M:%S",
        level=logging.INFO
    )

In [49]:
logger.info("Training/evaluation processs")

01/03/2023 14:15:16 - INFO - __main__ -   Training/evaluation processs


In [71]:
import json

fine_tune_model = model
fine_tune_tokenizer = tokenizer
fine_tune_train_dataset = train_dataset
fine_tune_val_dataset = val_dataset

args_dict = {
  "model_name_or_path": 't5-base',
  "max_len": 512 ,
  "target_max_len": 16,
  "output_dir": './models/tpu',
  "overwrite_output_dir": True,
  "per_gpu_train_batch_size": 8,
  "per_gpu_eval_batch_size": 8,
  "gradient_accumulation_steps": 4,
  "learning_rate": 1e-4,
  "tpu_num_cores": 8,
  "num_train_epochs": 4,
  "do_train": True
}

with open('args.json', 'w') as f:
  json.dump(args_dict, f)

hfparser = HfArgumentParser((ModelArguments, DataTrainingArguments, TrainingArguments))
model_args, data_args, training_args = hfparser.parse_json_file(json_file=os.path.abspath('args.json'))

if (
    os.path.exists(training_args.output_dir)
    and os.listdir(training_args.output_dir)
    and training_args.do_train
    and not training_args.overwrite_output_dir
):
    raise ValueError(
        f"Output directory ({training_args.output_dir}) already exists and is not empty. Use --overwrite_output_dir to overcome."
    )

logging.basicConfig(
      format="%(asctime)s - %(levelname)s - %(name)s -   %(message)s",
      datefmt="%m/%d/%Y %H:%M:%S",
      level=logging.INFO if training_args.local_rank in [-1, 0] else logging.WARN,
  )

logger.warning(
    "Process rank: %s, device: %s, n_gpu: %s, distributed training: %s, 16-bits training: %s",
    training_args.local_rank,
    training_args.device,
    training_args.n_gpu,
    bool(training_args.local_rank != -1),
    training_args.fp16,
)
logger.info("Training/evaluation parameters %s", training_args)

set_seed(training_args.seed)



PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
01/03/2023 15:14:15 - WARNING - __main__ -   Process rank: -1, device: cpu, n_gpu: 0, distributed training: False, 16-bits training: False
01/03/2023 15:14:15 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(
_n_gpu=0,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=False,
do_predict=False,
do_train=True,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
ev

In [74]:
""" 
Trace the model according to the data collator

'input_ids': input_ids, 
'attention_mask': attention_mask,
'lm_labels': lm_labels, 
'decoder_attention_mask': decoder_attention_mask
"""
traced_model = torch.jit.trace(func=fine_tune_model, example_inputs=(
    
))

c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\modeling_utils.py:734: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if causal_mask.shape[1] < attention_mask.shape[1]:


RuntimeError: Tracer cannot infer type of Seq2SeqLMOutput(loss=None, logits=tensor([[[-65.7776, -15.1142, -28.7621,  ..., -81.6770, -81.8706, -81.9388],
         [-41.7584,  -6.0590, -17.8186,  ..., -50.3439, -50.4415, -50.4755],
         [-18.5064,  -5.2212, -11.1655,  ..., -40.1127, -40.0975, -40.1528],
         ...,
         [-22.1067,  -6.7866, -12.6071,  ..., -44.5598, -44.5704, -44.6414],
         [-22.1067,  -6.7866, -12.6071,  ..., -44.5598, -44.5704, -44.6414],
         [-21.4868,  -6.6022, -12.3518,  ..., -44.0760, -44.0847, -44.1546]]],
       grad_fn=<UnsafeViewBackward0>), past_key_values=((tensor([[[[ 1.3385e+00, -2.1127e-02, -6.5480e-01,  ...,  9.4214e-02,
            7.5609e-03, -3.3276e-01],
          [ 8.8215e-01,  3.2993e-01, -1.1180e+00,  ..., -4.5154e-01,
           -1.8130e-01, -1.0115e+00],
          [-4.8649e-01, -2.3323e+00, -1.1428e+00,  ..., -2.5693e+00,
           -1.7539e+00, -5.6927e-01],
          ...,
          [-4.8649e-01, -2.3323e+00, -1.1428e+00,  ..

In [66]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=T5DataCollator(),
)

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


In [67]:
if training_args.do_train:
    trainer.train(
        model_path=model_args.model_name_or_path if os.path.isdir(model_args.model_name_or_path) else None
    )
    trainer.save_model()
    # For convenience, we also re-save the tokenizer to the same directory,
    # so that you can share your model easily on huggingface.co/models =)
    if trainer.is_world_master():
        tokenizer.save_pretrained(training_args.output_dir)

c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\trainer.py:1488: FutureWarning: `model_path` is deprecated and will be removed in a future version. Use `resume_from_checkpoint` instead.
  warnings.warn(
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1990
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train bat

  0%|          | 0/248 [00:00<?, ?it/s]

c:\Users\Popescu Andrei\anaconda3\envs\playground\lib\site-packages\transformers\models\t5\tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: target_attention_mask, target_ids. If target_attention_mask, target_ids are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds